# Zeeman slower
This document provides all the information about how to design a Zeeman slower using a magnetic field.
Includes an optional interactive matplotlib widget that lets you vary key parameters with sliders.

## Theory introduction


Here, we assume we are slowing down atoms at a constant deceleration using a counter-propagating laser beam at a rate $\frac{dv}{dt} = v \frac{dv}{dz} = - a$, and when we integrate over position we get the velocity profile as a function of position. Assuming constant deceleration, we can express the velocity as a function of position z along the slower as:
$$v(z) = \sqrt{v_0^2 - 2 a z}$$
where $v_0$ is the initial velocity of the atoms at the entrance of the slower, and $a$ is the constant deceleration which typically is $a=a_{max}/2$.
Thus, the stopping distance L required to slow atoms from initial velocity $v_0$ to final velocity $v_f$ is given by:
$$L = \frac{v_0^2 - v_f^2}{2 a}$$

Assuming $v_f=0$ we can redefine $v(z)$ in terms of L as:
$$v(z) = v_0 \sqrt{1 - \frac{z}{L}}$$

When a beam of particles is slowed down using a counter-propagating laser beam, the particles experience a changing Doppler shift as they decelerate (or a chirped laser $\omega_0$). To maintain resonance between the laser frequency and the atomic transition, a spatially varying magnetic field is applied along the length of the slower.

This magnetic field induces a Zeeman shift in the atomic energy levels, compensating for the changing Doppler shift.
$$\omega_0 + \frac{\mu_B B(z)}{\hbar} = \omega_L + k v(z)$$
where $\omega_0$ is the atomic transition frequency, $\omega_L$ is the laser frequency, $\mu_B$ is the Bohr magneton, $k$ is the wavevector of the laser light, and $v(z)$ is the velocity of the atoms at position z.
$$B(z)=B_0(1-z/L)^{1/2}+B_{bias}$$
where $B_0 = \frac{\hbar k v_0}{\mu_B} = \frac{h v_0}{\lambda \mu_B}$ is the maximum magnetic field strength at the entrance of the slower, and $B_{bias}$ is a constant offset magnetic field, such that $\mu_B B_{bias} = \hbar \Delta$ where $\Delta = \omega_L - \omega_0$ is the detuning of the laser frequency from the atomic transition frequency.

In case of need to look for more spectral lines, please refer to: https://physics.nist.gov/PhysRefData/ASD/lines_form.html

Or atoms information Safranova 
https://www1.udel.edu/atom

In [21]:
%matplotlib inline

import math
import wave
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as w
from IPython.display import display, clear_output

# Physical constants
hh = 6.62607015e-34  # J*s
hbar = hh / (2.0 * math.pi)
mu_B = 9.274009994e-24  # J/T
amu = 1.66053906660e-27  # kg


def compute_profile(mass_amu=40.078, wavelength=422.67276e-9, gamma=2*math.pi*34.6e6, g_eff=1.0,
                    v0=500.0, vf=50.0, s0=5.0, eta=0.5, n_points=500):
    m = mass_amu * amu
    k = 2 * math.pi / wavelength
    F_max = hbar * k * gamma / 2.0 * (s0 / (1.0 + s0))
    a_max = F_max / m
    a = eta * a_max
    if a <= 0:
        raise ValueError("Computed deceleration <= 0; check s0 and eta")
    if v0 <= vf:
        raise ValueError("Initial velocity v0 must be > final velocity vf")
    L = (v0*v0 - vf*vf) / (2.0*a)
    z = np.linspace(0.0, L, n_points)
    v = np.sqrt(np.maximum(0.0, v0*v0 - 2.0*a*z))
    B = 1 - (hbar * k * (v - vf)) / mu_B
    return z, v, B, L


def plot_profile(z, v, B, L):
    fig, ax1 = plt.subplots(figsize=(7,4))
    ax1.set_xlabel("z (m)")
    ax1.set_ylabel("B (G)", color='tab:blue')
    ax1.plot(z, B*1e4, color='tab:blue', lw=2)
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax1.grid(True, ls='--', alpha=0.4)
    ax2 = ax1.twinx()
    ax2.set_ylabel("v (m/s)", color='tab:orange')
    ax2.plot(z, v, color='tab:orange', lw=2)
    ax2.tick_params(axis='y', labelcolor='tab:orange')
    ax1.set_title(f"Zeeman slower profile (L = {L:.3f} m)")
    plt.tight_layout()
    plt.show()

In [22]:
# Species dropdown with mass, wavelength (m), and gamma (rad/s)
species_data = {
    'Ca-40':   {'mass': 40.078,  'wavelength': 422.67276e-9, 'gamma': 2*math.pi*34.6e6},
    'Sr-88':   {'mass': 87.62,   'wavelength': 460.862e-9,    'gamma': 2*math.pi*32.0e6},
    'Rb-87':   {'mass': 86.909,  'wavelength': 780.241e-9,    'gamma': 2*math.pi*6.065e6},
    'Cs-133':  {'mass': 132.905, 'wavelength': 852.347e-9,    'gamma': 2*math.pi*5.234e6},
    'Na-23':   {'mass': 22.990,  'wavelength': 589.158e-9,    'gamma': 2*math.pi*9.79e6},
    'Yb-174':  {'mass': 173.045, 'wavelength': 398.911e-9,    'gamma': 2*math.pi*29.0e6},
}

species = w.Dropdown(
    description='Species',
    options=list(species_data.keys()),
    value='Ca-40'
)

# Sliders (continuous_update=False avoids flicker & overdraw)
v0 = w.FloatSlider(description="v0 (m/s)", value=500.0, min=0.0, max=1200.0, step=5.0, continuous_update=False)
vf = w.FloatSlider(description="vf (m/s)", value=50.0, min=0.0, max=600.0, step=5.0, continuous_update=False)
eta = w.FloatSlider(description="eta", value=0.5, min=0.01, max=1.0, step=0.01, continuous_update=False)
s0 = w.FloatSlider(description="s0", value=5.0, min=0.1, max=50.0, step=0.1, continuous_update=False)
g_eff = w.FloatSlider(description="g_eff", value=1.0, min=0.1, max=2.0, step=0.01, continuous_update=False)
points = w.IntSlider(description="points", value=600, min=200, max=2000, step=50, continuous_update=False)

# Info display widgets
wavelength_label = w.HTML()
gamma_label = w.HTML()

# Output area
out = w.Output()

def update_info():
    d = species_data[species.value]
    wavelength_label.value = f"<b>λ</b>: {d['wavelength']*1e9:.3f} nm"
    gamma_label.value = f"<b>Γ/2π</b>: {d['gamma']/(2*math.pi)/1e6:.3f} MHz"

# Update logic (uses .observe for each slider)

def refresh(change=None):
    with out:
        clear_output(wait=True)
        d = species_data[species.value]
        update_info()
        try:
            z, v, B, L = compute_profile(mass_amu=d['mass'], wavelength=d['wavelength'],
                                         gamma=d['gamma'],
                                         v0=v0.value, vf=vf.value, eta=eta.value,
                                         s0=s0.value, g_eff=g_eff.value,
                                         n_points=points.value)
        except Exception as e:
            print("Parameter error:", e)
            return
        plot_profile(z, v, B, L)

for widget in (species, v0, vf, eta, s0, g_eff, points):
    widget.observe(refresh, names='value')

# UI container
ui = w.VBox([
    w.HBox([species, wavelength_label, gamma_label]),
    w.HBox([v0, vf]),
    w.HBox([eta, s0, g_eff]),
    points
])

# Initial render
refresh()

# Display (explicit display fixes plain text repr issues)
display(ui, out)

Output()

[comment]: # (Here, we assume we are slowing down atoms at a constant deceleration using a counter-propagating laser beam at a rate $\frac{dv}{dt} = -\eta a_{max}$, where $a_{max} = \frac{\hbar k \Gamma}{2 m} \frac{s_0}{1 + s_0}$ is the maximum achievable deceleration, $\eta$ is the design parameter (0 < $\eta$ < 1), $s_0$ is the on-resonance saturation parameter, and $\Gamma$ is the natural linewidth of the atomic transition.)